In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('train.csv')
data_test = pd.read_csv('test.csv')

In [3]:
data.pixels_y = data.pixels_y.astype('int64')

In [4]:
data_test.pixels_y = data_test.pixels_y.fillna(0)
data_test.pixels_y = data_test.pixels_y.astype('int64')

In [5]:
data.dtypes

id                       int64
name                    object
brand                   object
base_name               object
screen_size            float64
pixels_x                 int64
pixels_y                 int64
screen_surface          object
touchscreen              int64
cpu                     object
cpu_details             object
detachable_keyboard    float64
discrete_gpu             int64
gpu                     object
os                      object
os_details              object
ram                      int64
ssd                      int64
storage                  int64
weight                 float64
min_price              float64
max_price              float64
dtype: object

In [6]:
data_s = data[['id','brand', 'screen_size','pixels_y','screen_surface','touchscreen','cpu','discrete_gpu','gpu','os_details','ram','ssd','storage','weight','min_price','max_price']]
data_test_s = data_test[['id','brand', 'screen_size','pixels_y','screen_surface','touchscreen','cpu','discrete_gpu','gpu','os_details','ram','ssd','storage','weight']]

In [7]:
data_s

,id,brand,screen_size,pixels_y,screen_surface,touchscreen,cpu,discrete_gpu,gpu,os_details,ram,ssd,storage,weight,min_price,max_price
0,7774,Lenovo,15.6,1080,Glossy,1,Intel Core i7,0,Intel HD,Windows 10,8,0,1000,4.60,899.00,899.00
1,25926,Razer,15.6,1080,Matte,0,Intel Core i7,1,NVIDIA GeForce RTX 2070 Max-Q,Windows 10 Home,16,512,512,4.63,2099.99,2099.99
2,25267,HP,15.6,768,NaN,0,AMD A6,0,AMD Radeon R4,Windows 10,8,0,500,4.63,439.00,449.00
3,22367,Acer,15.6,1080,Matte,0,Intel Core i3,0,Intel UHD 620,Windows 10 Home,6,0,1000,5.30,375.00,449.00
4,17471,HP,17.3,900,Glossy,0,Intel Core i5,0,Intel HD 620,Windows 10,8,0,1000,5.80,559.00,559.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,3897,Dell,15.6,768,Glossy,1,Intel Core i3,0,Intel HD 4400,Windows 8.1,4,0,500,6.00,383.51,383.51
506,19301,Asus,17.3,1080,Matte,0,Intel Core i7,1,NVIDIA GeForce GTX 1070,Windows 10 Home,12,128,1128,6.39,1899.99,1899.99
507,22738,Asus,15.6,1080,Matte,0,Intel Core i5,0,Intel UHD 620,Windows 10 Home,8,128,1128,3.70,499.99,669.97
508,16468,HP,17.3,900,Glossy,0,Intel Core i5,0,Intel HD 620,Windows 10,8,0,1000,5.84,548.29,548.29


In [8]:
def process_onehot(df, list_values):
    transformed_df = pd.DataFrame()
    from sklearn import preprocessing
    for item in list_values:
        enc = preprocessing.OneHotEncoder()
        df[item] = df[item].astype(str)
        df[item] = df[item].str.lower()
        enc.fit(df[[item]].dropna())
        new_df = pd.DataFrame(data=enc.transform(df[[item]].dropna()).toarray(), columns=enc.categories_[0])
        transformed_df = pd.concat([transformed_df, new_df], axis=1)
    return transformed_df

In [9]:
processed_data = process_onehot(data, ['screen_surface'])
processed_data_test = process_onehot(data_test, ['screen_surface'])


In [10]:
data_f = pd.concat([processed_data,data_s[['id','pixels_y','ram','ssd','storage','weight', 'discrete_gpu','touchscreen','min_price','max_price']]], axis=1).set_index('id')
data_f_test = pd.concat([processed_data_test,data_test_s[['id','pixels_y','ram','ssd','storage','weight','touchscreen', 'discrete_gpu']]], axis=1).set_index('id').fillna(0)

In [11]:
data_f_test.shape

(222, 10)

In [12]:
data_f[data_f_test.columns]

,glossy,matte,nan,pixels_y,ram,ssd,storage,weight,touchscreen,discrete_gpu
id,,,,,,,,,,
7774,1.0,0.0,0.0,1080,8,0,1000,4.60,1,0
25926,0.0,1.0,0.0,1080,16,512,512,4.63,0,1
25267,0.0,0.0,1.0,768,8,0,500,4.63,0,0
22367,0.0,1.0,0.0,1080,6,0,1000,5.30,0,0
17471,1.0,0.0,0.0,900,8,0,1000,5.80,0,0
...,...,...,...,...,...,...,...,...,...,...
3897,1.0,0.0,0.0,768,4,0,500,6.00,1,0
19301,0.0,1.0,0.0,1080,12,128,1128,6.39,0,1
22738,0.0,1.0,0.0,1080,8,128,1128,3.70,0,0


In [18]:
from sklearn import decomposition
from sklearn import model_selection
from sklearn import ensemble
from sklearn import cluster
from sklearn import svm
from sklearn import linear_model
from sklearn import preprocessing

In [19]:

X_1 = data_f.iloc[:,0:-2].fillna(0)
y_1 = data_f.iloc[:,-2].fillna(0)
X_2 = data_f.iloc[:,0:-2].fillna(0)
y_2 = data_f.iloc[:,-1].fillna(0)
y_2

id
7774      899.00
25926    2099.99
25267     449.00
22367     449.00
17471     559.00
          ...   
3897      383.51
19301    1899.99
22738     669.97
16468     548.29
29633    1699.00
Name: max_price, Length: 510, dtype: float64

In [20]:
#clustering = cluster.FeatureAgglomeration(n_clusters=20)
#X_1 = clustering.fit_transform(X_1)
#X_2 = clustering.fit_transform(X_2)
#data_f_test_pp = clustering.fit_transform(data_f_test)

In [21]:
pca = decomposition.PCA(n_components=50)
#X_1 = pca.fit_transform(X_1)
#X_2 = pca.fit_transform(X_2)
#data_f_test_pp = pca.fit_transform(data_f_test)

In [22]:
pp = preprocessing.PowerTransformer(standardize=True)
X_1 = pp.fit_transform(X_1)
X_2 = pp.fit_transform(X_2)
data_f_test_pp = pp.fit_transform(data_f_test)

In [23]:
parameters = {'n_estimators':np.linspace(5,100,5).astype(int)}
model = linear_model.LinearRegression()
model2 = linear_model.LinearRegression()
#clf = model_selection.GridSearchCV(model, parameters)
#clf2 = model_selection.GridSearchCV(model2, parameters)

In [24]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_1,y_1, test_size=0.3, random_state=40 )
predictions = pd.DataFrame(index=y_test.index)

#clf.fit(X_train, y_train)
#model = clf.best_estimator_
model.fit(X_train, y_train)
predictions['min_pred'] = model.predict(X_test)
predictions['min_true'] = y_test
predictions['min_error'] = np.abs(predictions.min_pred-predictions.min_true)



In [25]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_2,y_2, test_size=0.3, random_state=40)
#clf.fit(X_train, y_train)
#model2 = clf.best_estimator_
model2.fit(X_train, y_train)
predictions['max_pred'] = model2.predict(X_test)
predictions['max_true'] = y_test
predictions['max_error'] = np.abs(predictions.max_pred-predictions.max_true)


In [26]:
print(predictions.min_error.mean())
print(predictions.max_error.mean())

224.88974424410236
229.6305368117248


In [27]:
np.mean(predictions.min_error+predictions.max_error)


454.52028105582707

In [28]:
y_train.shape

(357,)

In [29]:
predictions = pd.DataFrame(index=data_f_test.index)
model.fit(X_1,y_1)
model2.fit(X_2,y_2)
predictions['min_pred'] = model.predict(data_f_test)
predictions['max_pred'] = model2.predict(data_f_test)

In [30]:
predictions.to_csv('results.csv')

In [31]:
predictions

,min_pred,max_pred
id,,
28807,-3.905812e+15,-3.501651e+15
22559,-3.890593e+15,-3.488006e+15
28647,-3.890593e+15,-3.488006e+15
22141,-3.905812e+15,-3.501651e+15
26116,-3.890593e+15,-3.488006e+15
...,...,...
8398,-3.890593e+15,-3.488006e+15
21211,-3.905812e+15,-3.501651e+15
22553,-3.890593e+15,-3.488006e+15


In [32]:
data_f

,glossy,matte,nan,pixels_y,ram,ssd,storage,weight,discrete_gpu,touchscreen,min_price,max_price
id,,,,,,,,,,,,
7774,1.0,0.0,0.0,1080,8,0,1000,4.60,0,1,899.00,899.00
25926,0.0,1.0,0.0,1080,16,512,512,4.63,1,0,2099.99,2099.99
25267,0.0,0.0,1.0,768,8,0,500,4.63,0,0,439.00,449.00
22367,0.0,1.0,0.0,1080,6,0,1000,5.30,0,0,375.00,449.00
17471,1.0,0.0,0.0,900,8,0,1000,5.80,0,0,559.00,559.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3897,1.0,0.0,0.0,768,4,0,500,6.00,0,1,383.51,383.51
19301,0.0,1.0,0.0,1080,12,128,1128,6.39,1,0,1899.99,1899.99
22738,0.0,1.0,0.0,1080,8,128,1128,3.70,0,0,499.99,669.97
